## Imports & Global Vars

In [1]:
from Nim import *

Gene = namedtuple("Gene", ["condition","action","place"])
Individual = namedtuple("Individual", ["genome", "fitness"])

nim_rows = 5
eval_amount = 2000
genome_size = 3
population_size = 20
offspring_size = 50
generations = 10
mutation_rate = 0.5

## Conditions

In [2]:
def even_elems_if(state: Nim) -> bool:
    return sum(state.rows)%2 == 0

def odd_elems_if(state: Nim) -> bool:
    return sum(state.rows)%2 != 0

def even_stacks_if(state: Nim) -> bool:
    return len(state.rows)%2 == 0

def odd_stacks_if(state: Nim) -> bool:
    return len(state.rows)%2 != 0

def nimsum_if(state: Nim) -> bool:
    return nimSum(state.rows)!=0 

In [3]:
conditions = {"even_elems": even_elems_if, "odd_elems": odd_elems_if, "even_stacks": even_stacks_if, "odd_stacks": odd_stacks_if, "nimsum": nimsum_if}

## Actions

In [4]:
def all_get(state: Nim, row: int) -> Nimply:
    num_objects = state.rows[row]
    return Nimply(row, num_objects)

def one_get(state: Nim, row: int) -> Nimply:
    num_objects = 1
    return Nimply(row, num_objects)

def half_get(state: Nim, row: int) -> Nimply:
    num_objects = max(1,state.rows[row]//2)
    return Nimply(row, num_objects)

def rand_get(state: Nim, row: int) -> Nimply:
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def nimsum_get(state: Nim, row: int) -> Nimply:
    rows = state.rows
    totNimSum=nimSum(rows)

    if checkMisere(rows) != -1:
        if len(rows) % 2 == 0:
            return Nimply(row, rows[row])
        elif rows[row] > 1:
            return Nimply(row, rows[row]-1)

    lineNimSum=nimSum([totNimSum, rows[row]])
    if (lineNimSum < rows[row]):
        return Nimply(row, rows[row] - lineNimSum)
    ply = rand_get(state,row)
    return ply

In [5]:
actions = {"all": all_get, "one": one_get, "half": half_get, "rand": rand_get, "nimsum": nimsum_get}

## Places

In [6]:
def rand_place(state: Nim) -> int:
    row = random.randrange(0,len(state.rows))
    #print('rand place choose row: ', state, row)
    return row

def most_place(state: Nim) -> int:
    row = state.rows.index(max(state.rows))
    #print('most place choose row: ', state, row)
    return row

def least_place(state: Nim) -> int:
    row = state.rows.index(min([c for r, c in enumerate(state.rows) if c > 0]))
    return row

def nimsum_place(state: Nim) -> int:
    rows=state.rows
    totNimSum=nimSum(rows)
    if(totNimSum!=0):
        nonOneRow=checkMisere(rows)
        if nonOneRow!=-1:
            return nonOneRow
        for i,row in enumerate(rows):
            lineNimSum=nimSum([totNimSum,row])
            if(lineNimSum<row):
                return i
    return rand_place(state)

In [7]:
places = {"rand": rand_place, "most": most_place, "least": least_place, "nimsum": nimsum_place}

## Full Policy

In [8]:
def ga_play(state: Nim, genome: list) -> Nimply:
    for g in genome:
        if (conditions[g[0]](state)):
            move = actions[g[1]](state, places[g[2]](state))
            return move
    return lose_game()

class EvolvedAgent:
    def __init__(self, individual: Individual):
        self.genome = individual.genome
    def play(self, game):
        return ga_play(game, self.genome)

## Fitness

In [9]:
def evaluate(genome: list) -> float:
    win_count = 0
    for _ in range(eval_amount):
        game = Nim(random.randint(5,21))
        turn=1
        while not game.endTest():
            turn = 1 - turn
            if not turn:
                game.nimming(ga_play(game,genome))
            else:
                game.nimming(pure_random(game))
        if turn:
            win_count += 1
    return win_count/eval_amount

## GA Functions

In [10]:
import copy

def generate_population(population_size):
    population = list()
    for _ in range(population_size):
        genome = list()
        for _ in range(genome_size):
            gene = (random.choice(list(conditions.keys())), random.choice(list(actions.keys())), random.choice(list(places.keys())))
            genome.append(gene)
        population.append(Individual(genome, evaluate(genome)))
    return population

def mutation(indiv: Individual) -> Individual:
    mutable = [conditions, actions, places]

    mut_gene_idx = random.randrange(0, genome_size)
    mut_attr_idx = random.randrange(0, 3)
    attr = indiv.genome[mut_gene_idx][mut_attr_idx]

    acceptable = list(mutable[mut_attr_idx].keys())
    acceptable.remove(attr)
    new_attr = random.choice(acceptable)
    new_genome = copy.deepcopy(indiv.genome)
    new_genome[mut_gene_idx]=indiv.genome[mut_gene_idx][:mut_attr_idx]+(new_attr,)+indiv.genome[mut_gene_idx][mut_attr_idx+1:]
    fitness = evaluate(new_genome)
    return Individual(new_genome,fitness)

def cross_over(i1: Individual, i2: Individual) -> Individual:
    cross_over_point = random.randrange(1, genome_size)
    new_genome = list()
    for i in range(genome_size):
        if (i < cross_over_point):
            new_genome.append(i1.genome[i])
        else:
            new_genome.append(i2.genome[i])
    return Individual(new_genome, evaluate(new_genome))

def tournament(population,tournament_size=2):
    return max(random.choices(population, k=tournament_size), key=lambda i:i[1])

def evolution() -> Individual:
    population = generate_population(population_size)

    for g in range(generations):
        offspring = list()
        for i in range(offspring_size):
            if random.random() < 0.7:
                p = tournament(population)
                o = mutation(p)
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = cross_over(p1, p2)
            offspring.append(o)
        population += offspring
        population = sorted(population, key=lambda i: i[1], reverse=True)[:population_size]

    solution = population[0]
    return solution

## Play

In [11]:
game = Nim(nim_rows)
best_individual = evolution()
evaluate(best_individual.genome)

ea = EvolvedAgent(best_individual)
sandbox(game, ea.play)

ValueError: empty range for randrange() (1, 1, 0)